In [4]:
import torch 
import math

In [55]:
class Quantization:
    def __init__(self,quant_params):
        self.quant_params = quant_params
    @staticmethod
    def clamping(quant_params : torch.Tensor,lower_bound: int,upper_bound: int)->torch.Tensor:
        quant_params[quant_params < lower_bound] = lower_bound
        quant_params[quant_params > upper_bound] = upper_bound
        return quant_params
    def asym_quant(self,lower_bound=0,upper_bound=(2**8)-1)-> torch.Tensor:
        alpha = torch.max(self.quant_params)
        beta = torch.min(self.quant_params)
        self.scale_asym = (alpha - beta) / upper_bound
        self.zero_v = torch.round(-beta/self.scale_asym)
        self.asym_quant = Quantization.clamping(self.quant_params/self.scale_asym + self.zero_v,lower_bound,upper_bound).int()
        return self.asym_quant
    def asym_quant_perc(self,lower_bound=0,upper_bound=255,percentile : float=99.99):
        alpha = torch.percentile(self.quant_params,percentile)
        beta  = torch.percentile(self.quant_params,100-percentile)
        self.scale_asym_per = (alpha-beta)/upper_bound
        self.zero_per = torch.round(-beta/self.scale_asym_per)
        self.asym_quant_per = Quantization.clamping(self.quant_params/self.scale_asym_per+self.zero_per,lower_bound,upper_bound).int()
    def sym_quant(self,lower_bound=-(2**7)+1,upper_bound=(2**7)-1)-> torch.Tensor:
        alpha = torch.max(torch.abs(self.quant_params))
        scale = alpha / upper_bound
        self.scale_sym = scale
        quant_params = Quantization.clamping(self.quant_params/scale,lower_bound,upper_bound)
        self.sym_quant = quant_params.int()
        return self.sym_quant
    def asym_dequant(self)->torch.Tensor:
        return self.scale_asym * (self.asym_quant - self.zero_v)
    def sym_dequant(self):
        return self.scale_sym * self.sym_quant
    def asym_dequant_per(self):
        return self.scale_asym_per * (self.asym_quant_per - self.zero_per)
    
        
        
        